In [ ]:
from bs4 import BeautifulSoup 
import requests 
import csv

In [ ]:
name = input("Job name:\n").replace(" ","-")
city = input("City name:\n")
state = input("State name:\n")

Job name:
Receptionist
City name:
Mumbai
State name:
Maharashtra


In [ ]:
res = requests.get(f'https://www.indeed.co.in/{name}-jobs-in-{city},-{state}')
print(res)

<Response [200]>


In [ ]:
soup = BeautifulSoup(res.text,'html.parser')

In [ ]:
job_name = soup.select('.jobtitle')

In [ ]:
with open('jobs.csv', 'a',newline="\n",encoding="utf-8") as f:
                    writer = csv.writer(f)
                    writer.writerow(["title", "organization", "location", "salary", "url"])
for i in range(len(job_name)):
    job_name = soup.select('.jobtitle')[i].getText().strip()
    job_company = soup.select('.company')[i].getText().strip().replace(","," ")
    job_location = soup.select('.location')[i].getText().strip().replace(","," ")
    try :
        job_money = soup.select(".salaryText")[i].getText()
    except :
        job_money = ""
    h2 = soup.find('h2',{'class':'title'})
    a = h2.find('a')
    link = a.attrs.get("href")
    url = "https://www.indeed.co.in"+link
    row = [job_name,job_company,job_location,job_money,url]
    #print(row)
    with open('jobs.csv', 'a',newline="\n",encoding="utf-8") as f:
                    writer = csv.writer(f)
                    writer.writerow(row)


In [ ]:
import pandas as pd
import numpy as np
import json
import os
import re

import firebase_admin
from firebase_admin import firestore, credentials

# Generating Json

In [ ]:
def jsonify_row(r):
    r['salary'] = re.sub(r'[^\x00-\x7f]',r'', r['salary'])
    out = {
        'title' : r['title'],
        'organization' : r['organization'],
        'location' : r['location'],
        'salary' : r['salary'],
        'url' : r['url']
    }

    return out

In [ ]:
df = pd.read_csv('jobs.csv')

In [ ]:
df.head()

,title,organization,location,salary,url,json
0,Corporate Receptionist,Dirt Busters,Mumbai Maharashtra,"1,50,000 - 2,50,000 a year",https://www.indeed.co.in/rc/clk?jk=f16737efa4c...,"{'title': 'Corporate Receptionist', 'organizat..."
1,Receptionist Job For Freshers,Magna India Services,Mumbai Maharashtra,"1,50,000 - 2,50,000 a year",https://www.indeed.co.in/rc/clk?jk=f16737efa4c...,"{'title': 'Receptionist Job For Freshers', 'or..."
2,Receptionist Job In Bengaluru,Magna India Services,Mumbai Maharashtra,"1,00,000 - 3,00,000 a year",https://www.indeed.co.in/rc/clk?jk=f16737efa4c...,"{'title': 'Receptionist Job In Bengaluru', 'or..."
3,Senior Receptionist,Mark IT Solutions,Mumbai Maharashtra,"4,000 - 6,000 a month",https://www.indeed.co.in/rc/clk?jk=f16737efa4c...,"{'title': 'Senior Receptionist', 'organization..."
4,Receptionist / Front Desk Manager,Flexiventures Private Limited,Andheri Mumbai Maharashtra,-,https://www.indeed.co.in/rc/clk?jk=f16737efa4c...,"{'title': 'Receptionist / Front Desk Manager',..."


In [ ]:
df.salary = df.salary.fillna('-')
df.salary = df.salary.str.strip()
df['json'] = df.apply(lambda x : jsonify_row(x), axis=1)

In [ ]:
cred = credentials.Certificate("secret.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

ValueError: ignored

In [ ]:
for job in df.json.values:
    db.collection('indeedJobs').add(job)